In [1]:
import pandas as pd
from pathlib import Path

In [2]:
path = Path.cwd()
file_dir = Path.cwd()/'data'

In [3]:
train_df = pd.read_csv(file_dir/'train.csv')
train_df.drop(314, inplace = True)
train_df['text'] = train_df['text'].apply(lambda x: str(x))
train_df['sentiment'] = train_df['sentiment'].apply(lambda x: str(x))
train_df['selected_text'] = train_df['selected_text'].apply(lambda x: str(x))

In [4]:
test_df = pd.read_csv(file_dir/'test.csv')
test_df['text'] = test_df['text'].apply(lambda x: str(x))
test_df['sentiment'] = test_df['sentiment'].apply(lambda x: str(x))

In [5]:
# test_df['old_sentiment'] = pd.merge(test_df, other_df, left_on='textID', right_on='aux_id', how='left').sentiment_y

In [6]:
desktop = Path('/home/jack/Desktop')

In [7]:
other_df = pd.read_csv(desktop/'tweet_dataset.csv')

In [8]:
# train_df['old_sentiment'] = pd.merge(train_df, other_df, left_on='textID', right_on='aux_id', how='left').sentiment_y

In [9]:
train_df = train_df.merge(other_df[['aux_id','sentiment']], left_on='textID', right_on='aux_id', how='left')
train_df.drop('aux_id', axis = 1, inplace = True)

In [10]:
train_df

,textID,text,selected_text,sentiment_x,sentiment_y
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sadness
2,088c60f138,my boss is bullying me...,bullying me,negative,worry
3,9642c003ef,what interview! leave me alone,leave me alone,negative,worry
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,hate
...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,worry
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,sadness
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,relief
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,relief


In [11]:
train_df['new_sentiment'] = train_df.sentiment_x + ' ' + train_df.sentiment_y

In [12]:
import transformers
from transformers import AutoModelForQuestionAnswering, AutoModel, AutoConfig, get_linear_schedule_with_warmup
from transformers.optimization import AdamW
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from pathlib import Path
import os
from itertools import compress
import utils
from sklearn.model_selection import StratifiedKFold

In [13]:
from torch.utils.data import DataLoader
from apex.optimizers.fused_lamb import FusedLAMB as Lamb
from ranger import Ranger
from fairseq import criterions 
from functools import partial
from tokenizers import BertWordPieceTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from fastai.core import *
from fastai.text import *

In [14]:
max_len = 128
bs = 32
tokenizer = transformers.RobertaTokenizerFast('input/roberta-base/vocab.json', 'input/roberta-base/merges.txt',
                                  lowercase=True,add_prefix_space=True).tokenizer

In [15]:
def preprocess(sentiment, tweet, selected, tokenizer, max_len):
    _input = tokenizer.encode(sentiment, tweet)
    _span = tokenizer.encode(selected, add_special_tokens=False)
    
    len_span = len(_span.ids)
    start_idx = None
    end_idx = None
    
    for ind in (i for i, e in enumerate(_input.ids) if e == _span.ids[0]):
        if _input.ids[ind: ind + len_span] == _span.ids:
            start_idx = ind
            end_idx = ind + len_span - 1
            break
    
    # Handles cases where tokenizing input & span separately produces different outputs
    if not start_idx:
        idx0 = tweet.find(selected)
        idx1 = idx0 + len(selected)
        
        char_targets = [0] * len(tweet)
        if idx0 != None and idx1 != None:
            for ct in range(idx0, idx1):
                char_targets[ct] = 1
                
        tweet_offsets = _input.offsets[5:-1]
        
        target_idx = []
        for j, (offset1, offset2) in enumerate(tweet_offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)
                
        start_idx, end_idx = target_idx[0] + 5 , target_idx[-1] + 5
        
    _input.start_target = start_idx
    _input.end_target = end_idx
    _input.tweet = tweet
    _input.sentiment = sentiment
    _input.selected = selected
    
    _input.pad(max_len)
    
    return _input

In [16]:
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss    

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, ε:float=0.1, reduction='mean'):
        super().__init__()
        self.ε,self.reduction = ε,reduction
    
    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return torch.lerp(nll, loss/c, self.ε) 

In [17]:
class TweetDataset(Dataset):
    def __init__(self, dataset, test = None):
        self.df = dataset
        self.test = test
        
    def __getitem__(self, idx):
        if not self.test:
            sentiment, tweet, selected = (self.df[col][idx] for col in ['new_sentiment', 'text', 'selected_text'])
            _input = preprocess(sentiment, tweet, selected, tokenizer, max_len)
            
            yb = [torch.tensor(_input.start_target), torch.tensor(_input.end_target)]
            
        else:
            _input = tokenizer.encode(self.df.sentiment[idx], self.df.text[idx])
            _input.pad(max_len)
            
            yb = 0

        xb = [torch.LongTensor(_input.ids),
              torch.LongTensor(_input.attention_mask),
              np.array(_input.offsets)]

        return xb, yb     

    def __len__(self):
        return len(self.df)

In [18]:
roberta_path = 'input/roberta-base'
pt_model = AutoModel.from_pretrained(roberta_path)

INFO:transformers.configuration_utils:loading configuration file input/roberta-base/config.json
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 1

In [19]:
class SpanModel(nn.Module):
    def __init__(self,pt_model):
        super().__init__()
        self.model = pt_model
        self.drop_out = nn.Dropout(0.1)
        self.qa_outputs1c = torch.nn.Conv1d(768*2, 128, 2)
        self.qa_outputs2c = torch.nn.Conv1d(768*2, 128, 2)
        self.qa_outputs1 = nn.Linear(128, 1)
        self.qa_outputs2 = nn.Linear(128, 1)
        
#         self.qa_outputs = nn.Linear(768 * 2, 2) # update hidden size

    # could pass offsets here and not use - can grab in last_input
    def forward(self, input_ids, attention_mask, offsets = None):
        
        _, _, hidden_states = self.model(input_ids, attention_mask=attention_mask)

        out = torch.cat((hidden_states[-1], hidden_states[-2]), dim=-1)
        out = self.drop_out(out)
        out = torch.nn.functional.pad(out.transpose(1,2), (1, 0))
        
        out1 = self.qa_outputs1c(out).transpose(1,2)
        out2 = self.qa_outputs2c(out).transpose(1,2)

        start_logits = self.qa_outputs1(self.drop_out(out1)).squeeze(-1)
        end_logits = self.qa_outputs2(self.drop_out(out2)).squeeze(-1)
        return start_logits, end_logits

In [20]:
def dist_between(start_logits, end_logits, device='cpu', max_seq_len=128):
    """get dist btw. pred & ground_truth"""

    linear_func = torch.tensor(np.linspace(0, 1, max_seq_len, endpoint=False), requires_grad=False)
    linear_func = linear_func.to(device)

    start_pos = (start_logits*linear_func).sum(axis=1)
    end_pos = (end_logits*linear_func).sum(axis=1)

    diff = end_pos-start_pos

    return diff.sum(axis=0)/diff.size(0)


def dist_loss(start_logits, end_logits, start_positions, end_positions, device='cpu', max_seq_len=128, scale=1):
    """calculate distance loss between prediction's length & GT's length
    
    Input
    - start_logits ; shape (batch, max_seq_len{128})
        - logits for start index
    - end_logits
        - logits for end index
    - start_positions ; shape (batch, 1)
        - start index for GT
    - end_positions
        - end index for GT
    """
    start_logits = torch.nn.Softmax(1)(start_logits) # shape ; (batch, max_seq_len)
    end_logits = torch.nn.Softmax(1)(end_logits)
    
    start_one_hot = torch.nn.functional.one_hot(start_positions, num_classes=max_seq_len).to(device)
    end_one_hot = torch.nn.functional.one_hot(end_positions, num_classes=max_seq_len).to(device)
    
    pred_dist = dist_between(start_logits, end_logits, device, max_seq_len)
    gt_dist = dist_between(start_one_hot, end_one_hot, device, max_seq_len) # always positive
    diff = (gt_dist-pred_dist)

    rev_diff_squared = 1-torch.sqrt(diff*diff) # as diff is smaller, make it get closer to the one
    loss = -torch.log(rev_diff_squared) # by using negative log function, if argument is near zero -> inifinite, near one -> zero

    return loss*scale

In [21]:
class join_loss(Module):
    def __init__(self, ce_loss_fn = nn.CrossEntropyLoss(), device = 'cpu'): 
        self.loss_fn = ce_loss_fn
        self.device = device
        
    def forward(self, inputs, start_targets, end_targets):
        start_logits, end_logits = inputs # assumes tuple input
        
        logits = torch.cat([start_logits, end_logits]).contiguous()
        
        targets = torch.cat([start_targets, end_targets]).contiguous()
        
        ce_loss = self.loss_fn(logits, targets)
        
        distance_loss = dist_loss(start_logits, end_logits, 
                                  start_targets, end_targets, device=device, max_seq_len =128)
        return ce_loss+distance_loss

In [30]:
class CELoss(Module):
    def __init__(self, loss_fn = nn.CrossEntropyLoss()): 
        self.loss_fn = loss_fn
        
    def forward(self, inputs, start_targets, end_targets):
        start_logits, end_logits = inputs # assumes tuple input
        
        logits = torch.cat([start_logits, end_logits]).contiguous()
        
        targets = torch.cat([start_targets, end_targets]).contiguous()
        
        return self.loss_fn(logits, targets)

In [23]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [24]:
def get_best_start_end_idxs(start_logits, end_logits):
    max_len = len(start_logits)
    start_logits, end_logits = start_logits.cpu(),end_logits.cpu()
    
    a = np.tile(start_logits, (max_len, 1))
    b = np.tile(end_logits, (max_len, 1))
    c = np.tril(a + b.T, k=0).T
    c[c == 0] = -1000
    return np.unravel_index(c.argmax(), c.shape)

In [25]:
# Note that validation ds is by default not shuffled in fastai - so indexing like this will work for Callback
# https://forums.fast.ai/t/how-to-set-shuffle-false-of-train-and-val/33730

class JaccardScore(Callback):
    "Stores predictions and targets to perform calculations on epoch end."
    def __init__(self, valid_ds): 
        self.valid_ds = valid_ds
        self.context_text = valid_ds.df.text
        self.answer_text = valid_ds.df.selected_text
        
    def on_epoch_begin(self, **kwargs):
        self.jaccard_scores = []  
        self.valid_ds_idx = 0
        
        
    def on_batch_end(self, last_input:Tensor, last_output:Tensor, last_target:Tensor, **kwargs):
              
        input_ids = last_input[0]
        offsets = last_input[2]

        start_logits, end_logits = last_output
        
        # for id in batch of ids
        for i in range(len(input_ids)):
            
            _offsets = offsets[i]
#             start_idx, end_idx = torch.argmax(start_logits[i]), torch.argmax(end_logits[i])
            
            start_idx, end_idx = get_best_start_end_idxs(start_logits[i], end_logits[i])
            
            _answer_text = self.answer_text[self.valid_ds_idx]
            original_start, original_end = _offsets[start_idx][0], _offsets[end_idx][1]
            pred_span = self.context_text[self.valid_ds_idx][original_start : original_end]
                
            score = jaccard(pred_span, _answer_text)
            self.jaccard_scores.append(score)

            self.valid_ds_idx += 1
            
    def on_epoch_end(self, last_metrics, **kwargs):        
        res = np.mean(self.jaccard_scores)
        return add_metrics(last_metrics, res)

In [26]:
from fastai.callbacks import *

def new_on_train_begin(self, **kwargs:Any)->None:
    "Initializes the best value."
    if not hasattr(self, 'best'):
        self.best = float('inf') if self.operator == np.less else -float('inf')

SaveModelCallback.on_train_begin = new_on_train_begin

In [27]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [28]:
fold_indices = list(skf.split(train_df, train_df.new_sentiment))

In [31]:
device = torch.device('cuda')

def run_fold(fold):
    wd = 0.01 # high value but worked quite well
    lr = 1e-4
    
    tr_df  = train_df.iloc[fold_indices[fold][0]].reset_index(drop=True)
    val_df = train_df.iloc[fold_indices[fold][1]].reset_index(drop=True)
    
    train_ds, valid_ds = [TweetDataset(i) for i in [tr_df,val_df]]
    data = DataBunch.create(train_ds, valid_ds,bs = bs)
#     loss_fn = partial(CELoss, LabelSmoothingCrossEntropy())
    loss_fn = join_loss(LabelSmoothingCrossEntropy(), device) # no need to initialise later - no partial 

    pt_model = AutoModel.from_pretrained(roberta_path)
    model = SpanModel(pt_model)
    learner = Learner(data, model, loss_func = loss_fn, opt_func = Ranger, metrics = [JaccardScore(valid_ds)])
    
    early_stop_cb = EarlyStoppingCallback(learner, monitor='jaccard_score',mode='max',patience=2)
    save_model_cb = SaveModelCallback(learner,every='improvement',monitor='jaccard_score',name=f'roberta_conv_fold_{fold}')
    csv_logger_cb = CSVLogger(learner, f"roberta_conv_logs_{fold}", True)

    learner.fit_fc(4, 1e-4,start_pct=0.6, wd=wd, callbacks=[early_stop_cb, save_model_cb, csv_logger_cb])
    learner.fit_fc(3, 3e-5,start_pct=0.6, wd=wd, callbacks=[early_stop_cb, save_model_cb, csv_logger_cb])

    learner.save(f'roberta_conv_after_epoch_{fold}', with_opt=False)
    del learner; del pt_model; gc.collect()

In [ ]:
# Ranger FC RUN
for fold in range(1,5):run_fold(fold)

INFO:transformers.configuration_utils:loading configuration file input/roberta-base/config.json
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 1

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


epoch,train_loss,valid_loss,jaccard_score,time
0,1.605508,1.584496,0.699480,09:38
1,1.506651,1.513743,0.704634,10:03
2,1.436278,1.543316,0.695403,09:58
3,1.318321,1.512942,0.708374,10:55


Better model found at epoch 0 with jaccard_score value: 0.6994798050652977.
Better model found at epoch 1 with jaccard_score value: 0.7046335294632677.
Better model found at epoch 3 with jaccard_score value: 0.7083736517595187.
set state called


epoch,train_loss,valid_loss,jaccard_score,time
0,1.310541,1.533289,0.701220,09:43
1,1.276020,1.573133,0.692638,09:53
